<a href="https://colab.research.google.com/github/Devwarlt/BairesDev---Machine-Learning-Training/blob/main/treinamento_de_redes_neurais_com_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network -  `Transfer Learning`

In [ ]:
%matplotlib inline

import os
os.environ["KERAS_BACKEND"] = "tensorflow" # Theano with GPU

import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import keras

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential, Model
from keras.layers import (
    Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
    )